In [1]:
import numpy as np
from scipy.spatial import Delaunay
import numba
from numba import float32, float64, int64
from typing import Union
import illustris_python as il
import matplotlib.pyplot as plt

## Define the DTFE class and additional functions

In [2]:
@numba.jit(nopython=True, nogil=True)
def tetrahedron_volume(sim: int64[:], points: float64[:,:]):
    return abs(np.linalg.det(np.stack((points[sim[1]] - points[sim[0]], 
                                       points[sim[2]] - points[sim[0]],
                                       points[sim[3]] - points[sim[0]])))) / 6

@numba.jit(nopython=True, nogil=True)
def compute_densities(pts: float64[:,:], simps: float64[:,:],
                      m: Union[float64, float64[:]]) -> np.ndarray:
    M = len(pts)
    rho = np.zeros(M, dtype='float64')
    for sim in simps:
        vol = tetrahedron_volume(sim, pts)
        for index in sim:
            rho[index] += vol
    return (3 + 1) * m / rho

@numba.jit(nopython=True, nogil=True)
def compute_gradients(pts: float64[:,:], simps: float64[:,:], rho: float64[:],
                      v: float64[:,:]) -> tuple[np.ndarray, np.ndarray]:
    N = len(simps)
    Drho = np.zeros((N, 3), dtype='float64')
    Dv   = np.zeros((N, 3, 3), dtype='float64')

    for i, s in enumerate(simps):
        [p0, p1, p2, p3] = pts[s]
        [r0, r1, r2, r3] = rho[s]
        [v0, v1, v2, v3] = v[s]

        Ainv: float64[:,:] = np.linalg.inv(np.stack((p1 - p0, p2 - p0, p3 - p0)))
        Drho[i] = Ainv @ np.array([r1 - r0, r2 - r0, r3 - r0])
        Dv[i] = Ainv @ np.stack((v1 - v0, v2 - v0, v3 - v0))
    return (Drho, Dv)

@numba.jit(nopython=True, nogil=True)
def map_affine(a, b, c):
    assert(len(a) == len(b) == len(c))
    result = np.zeros_like(a)
    for i in range(len(a)):
        result[i] = a[i] + b[i] @ c[i]
    return result

#The Delaunay Tesselation Field Estimator 
class DTFE:
    def __init__(self, points, velocities, m):
        print("Delaunay Tesselation Field Estimator initialization:")
        self.velocities = velocities
        print("\t-Evaluate Delaunay tessellation")
        self.delaunay = Delaunay(points)
        
        #Area of a triangle
        
        #The density estimate
        print("\t-Evaluate density estimate")
        self.rho = compute_densities(self.delaunay.points, self.delaunay.simplices, m)
        #The gradients
        print("\t-Evaluate gradients")
        self.Drho, self.Dv = compute_gradients(self.delaunay.points, self.delaunay.simplices,
                                               self.rho, self.velocities)

    #The interpolations
    def density(self, x, y, z):
        simplexIndex = self.delaunay.find_simplex(np.c_[x, y, z])
        pointIndex   = self.delaunay.simplices[simplexIndex][...,0]
        return map_affine(self.rho[pointIndex], self.Drho[simplexIndex],
                          np.c_[x, y, z] - self.delaunay.points[pointIndex])

    def v(self, x, y, z):
        simplexIndex = self.delaunay.find_simplex(np.c_[x, y, z])
        pointIndex   = self.delaunay.simplices[simplexIndex][...,0]
        return map_affine(self.velocities[pointIndex], self.Dv[simplexIndex],
                          np.c_[x, y, z] - self.delaunay.points[pointIndex])
    
    def gradV(self, x, y, z):
        return self.Dv[self.delaunay.find_simplex(np.c_[x, y, z])]

    def theta(self, x, y, z):
        simplexIndex = self.delaunay.find_simplex(np.c_[x, y, z])
        return (self.Dv[simplexIndex][...,0,0] + 
                self.Dv[simplexIndex][...,1,1] + 
                self.Dv[simplexIndex][...,2,2])

    def sigma(self, x, y, z):
        simplexIndex = self.delaunay.find_simplex(np.c_[x, y, z])
        Dv = self.Dv[simplexIndex]
        theta = Dv[...,0,0] + Dv[...,1,1] + Dv[...,2,2]
        return np.array([[Dv[...,0,0] - theta / 3       , (Dv[...,0,1] + Dv[...,1,0]) / 2, (Dv[...,0,2] + Dv[...,2,0]) / 2],
                        [(Dv[...,1,0] + Dv[...,0,1]) / 2,  Dv[...,1,1] - theta / 3       , (Dv[...,1,2] + Dv[...,2,1]) / 2],
                        [(Dv[...,2,0] + Dv[...,0,2]) / 2, (Dv[...,2,1] + Dv[...,1,2]) / 2,  Dv[...,2,2] - theta / 3       ]]) 
    
    def omega(self, x, y, z):
        simplexIndex = self.delaunay.find_simplex(np.c_[x, y, z])
        Dv = self.Dv[simplexIndex]
        zeros = np.zeros(len(simplexIndex))
        return (np.array([[zeros, (Dv[...,0,1] - Dv[...,1,0]) / 2, (Dv[...,0,2] - Dv[...,2,0]) / 2],
                          [(Dv[...,1,0] - Dv[...,0,1]) / 2, zeros, (Dv[...,1,2] - Dv[...,2,1]) / 2],
                          [(Dv[...,2,0] - Dv[...,0,2]) / 2, (Dv[...,2,1] - Dv[...,1,2]) / 2, zeros]])) 

## Load snapshot

In [3]:
base_path = "/Users/users/nastase/PROJECT/"
snapshot_number = 133

In [4]:
dm_data = il.snapshot.loadSubset(base_path, snapshot_number,'dm', ['Coordinates', 'Velocities'])
dm_pos_all = dm_data['Coordinates']
dm_vel_all = dm_data['Velocities']

In [5]:
dm_data

{'count': 94196375,
 'Coordinates': array([[  851.8552  , 26336.646   , 18342.275   ],
        [  853.7293  , 26339.006   , 18343.473   ],
        [  850.2726  , 26341.738   , 18344.63    ],
        ...,
        [60515.91    , 49023.797   , 56935.59    ],
        [65718.586   , 61601.94    , 46827.504   ],
        [63968.37    , 61429.832   ,    72.121796]], dtype=float32),
 'Velocities': array([[   7.6957703, -183.39737  , -136.04996  ],
        [ 120.916016 , -130.1967   , -160.76692  ],
        [ 102.222404 ,  -92.88087  , -200.93481  ],
        ...,
        [  87.63591  ,  151.37038  , -178.47028  ],
        [ 245.2501   ,   24.462906 ,  117.35533  ],
        [ 179.87665  ,   17.334671 ,   -8.167839 ]], dtype=float32)}

## Explore data after loading

In [ ]:
dm_pos_all

In [ ]:
dm_vel_all

## Create a filter for the data

In [ ]:
x_min, x_max = 10_000, 25_000
y_min, y_max = 10_000, 25_000
z_min, z_max = 10_000, 25_000

In [ ]:
x_filter = (dm_pos_all[:,0] >= x_min) & (dm_pos_all[:,0] <= x_max)
y_filter = (dm_pos_all[:,1] >= y_min) & (dm_pos_all[:,1] <= y_max) 
z_filter = (dm_pos_all[:,2] >= z_min) & (dm_pos_all[:,2] <= z_max) 

In [ ]:
data_filter = x_filter & y_filter & z_filter

## Take a subset of data

In [ ]:
dm_pos_subset = dm_pos_all[data_filter].astype(np.float64)
dm_vel_subset = dm_vel_all[data_filter].astype(np.float64)

## Run DTFE

In [ ]:
m = np.ones(len(dm_pos_subset)).astype(np.float64)

In [ ]:
%%time
dtfe = DTFE(dm_pos_subset, dm_vel_subset, m)

## Compute densities

In [ ]:
L = x_max - x_min
n = 256

X, Y = np.meshgrid(np.arange(0.1 * L, 0.9 * L, 0.8 * L / n),   
                   np.arange(0.1 * L, 0.9 * L, 0.8 * L / n))

In [ ]:
%%time
dummy_coordinate = np.full_like(X, int(L / 2)).flat
dens  = dtfe.density(dummy_coordinate, Y.flat, X.flat).reshape((n,n))

## Explore dens

## Plot data

In [ ]:
def densPlot(data, imageSize):
    X = np.arange(0, data.shape[0])
    Y = np.arange(0, data.shape[1])
    X, Y = np.meshgrid(X, Y)

    plt.figure(figsize=(imageSize, imageSize))
    plt.pcolormesh(X, Y, data, shading='auto')
    plt.axis("equal")
    plt.show()

In [ ]:
delta = 125
densPlot(np.log(dens[(0 + delta):(255 - delta), (0 + delta):(255 - delta)]), 10)

## Plot data 3D

In [ ]:
L = x_max - x_min
n = 256

X, Y, Z = np.meshgrid(np.arange(0.1 * L, 0.9 * L, 0.8 * L / n),
                      np.arange(0.1 * L, 0.9 * L, 0.8 * L / n),
                      np.arange(0.1 * L, 0.9 * L, 0.8 * L / n))

dens  = dtfe.density(X.flat, Y.flat, Z.flat).reshape((n,n,n))

In [ ]:
def densPlot3D(data, imageSize):
    X = np.arange(0, data.shape[0])
    Z = np.arange(0, data.shape[2])
    X, Z = np.meshgrid(X, Z)

    plt.figure(figsize=(imageSize, imageSize))
    plt.pcolormesh(X, Z, data, shading='auto')
    plt.axis("equal")
    plt.show()

In [ ]:
densPlot3D(dens, 10)

## Plot data 3D from Ispirov

In [ ]:
n = 256
L = 75000
wid = 750

X,Y,Z = np.meshgrid(np.linspace(-L/2,L/2,n),np.linspace(-L/2,L/2,n),np.linspace(-wid/2,wid/2,n))

In [ ]:
d = dtfe.density(X.flatten(),Y.flatten(),Z.flatten())

In [ ]:
den = d.reshape(n,n,n)

In [ ]:
delta = 60
dens = den[:, delta:(n-delta), delta:(n-delta)]

In [ ]:
plt.imshow(dens[200,:,:],norm="log",origin="lower",vmin =1e-8,vmax = 1e-3 )
plt.yticks(np.linspace(0,n-2,5), np.linspace(-L/2,L/2,5,dtype=int))
plt.xticks(np.linspace(0,n-2,5), np.linspace(-L/2,L/2,5,dtype=int))

plt.title("Physical Space DTFE")
plt.colorbar()
plt.show()